In [1]:
import convokit

In [2]:
from convokit import Corpus, download

### Downloading dataset

In [3]:
de = Corpus(filename=download("subreddit-de"))
germany = Corpus(filename=download("subreddit-germany"))

Dataset already exists at /Users/nianyiwang/.convokit/downloads/subreddit-de
Dataset already exists at /Users/nianyiwang/.convokit/downloads/subreddit-germany


#### Getting common users

In [23]:
eng_users = set()
de_users=set()
for usr in de.iter_users():
    de_users.add(usr.name)
for usr in germany.iter_users():
    eng_users.add(usr.name)

In [45]:
common_users = list(eng_users.intersection(de_users))
print(len(common_users))
eng_only_users = list(eng_users.difference(de_users))
print(len(eng_only_users))

14072
43550


In [46]:
common_users[0]

'panda527'

#### Extract texts that bilingual users posted in English

In [48]:
user_conv = {}
for utt in germany.iter_utterances():
    usr = utt.user.name
    if usr not in common_users or usr == '[deleted]':
        continue
    if usr not in user_conv and utt.text != '':
        user_conv[usr] = [utt.text]
    elif usr in user_conv and utt.text != '':
        user_conv[usr].append(utt.text)

In [55]:
print(user_conv['panda527'])
print(len(user_conv))

['Come of you please give us some context?', 'It actually is, the lower plaque below the old ASU sticker is gone.', "True that - nothing really wants to kill you in German Forests... just make sure to follow any posted warnings for tree felling and you should be good.\nOh yeah and don't just eat any mushroom! Enjoy!", 'G’socks\ngrattler', 'I heard Congstar is great for such needs.', 'I would also add your education to this. If you are a hairdresser and make six figures with your own salon big props - if you are a automotive engineer in Munich or Stuttgart and are living in your parents basement at the age of 30 because you can’t afford to move out .. well then there is something off.']
13621


In [32]:
uttid_with_multi_lang_users = []
for usr in user_conv:
    for des in user_conv[usr]:
        uttid_with_multi_lang_users.append(des['id'])

In [33]:
print(uttid_with_multi_lang_users[:10])

['o12ea', '2u3z9n', 'c2sf8te', 'c2sfb3a', 'c2tz15c', 'c35r171', 'c37bmet', 'c37cc08', 'c37cwn7', 'c37g7u1']


In [34]:
eng_filtered_convs = []
for conv in germany.iter_conversations():
    for uttid in conv._utterance_ids:
        eng_filtered_convs.append(uttid)

In [35]:
print(eng_filtered_convs[:10])

['cmiy0', 'cmvu8', 'c0tpolx', 'c0tqmtk', 'c0ww9oz', 'cn62c', 'co951', 'cpsyd', 'c0uej3z', 'c0uemy0']


#### Constructing training set
12000 bilingual users and 12000 no cross-over users

In [56]:
import random
bi_users = list(user_conv.keys())
random.shuffle(bi_users)
bi_users = bi_users[:12000]
bi_utt = {}
for usr in bi_users:
    bi_utt[usr] = user_conv[usr]

In [58]:
sin_utts = {}
for utt in germany.iter_utterances():
    usr = utt.user.name
    if usr not in sin_users or usr == '[deleted]':
        continue
    if usr not in sin_utts and utt.text != '':
        sin_utts[usr] = [utt.text]
    elif usr in sin_utts and utt.text != '':
        sin_utts[usr].append(utt.text)
sin_users = list(sin_utts.keys())
random.shuffle(sin_users)
sin_users = sin_users[:12000]
sin_utt = {}
for usr in sin_users:
    sin_utt[usr] = sin_utts[usr]

In [60]:
print(bi_utt['I_Like_Spaghetti'])
print(sin_utt['EureMutter'])

['What do blondes and spaghetti have in common? They both wiggle when you eat them.', 'What do blondes and spaghetti have in common? They both wiggle when you eat them.', '(╯ಠ_ಠ）╯︵ ┻━┻', 'Did you hear about the Italian chef that died? He pasta way. ']
['Nice meme', 'Euro Truck Simulator 2']
